### 数据准备

In [11]:
import pandas as pd
# 数据加载
train_data = pd.read_csv("./data_file/titanic/train.csv")
test_data = pd.read_csv("./data_file/titanic/test.csv")
# 进行基本信息统计和筛选
print(train_data.info())
print("****"*10)
print(train_data.describe())
print("****"*10)
print(train_data.describe(include=['O']))
print("****"*10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
****************************************
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000 

### 数据清洗

> Age，Fare（船票价格），Cabin这三个字段缺失

In [14]:
# 使用平均值来填充年龄
train_data['Age'].fillna(train_data['Age'].mean(),inplace=True)
test_data["Age"].fillna(test_data["Age"].mean(),inplace =True)

# 使用票价的平均值填充nan值
train_data['Fare'].fillna(train_data['Fare'].mean(),inplace = True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace = True)

In [18]:
# 对Embarked字段的取值观察方法如下：
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [20]:
# 由于大部分都是S，我们就用s来填充Embarked
train_data["Embarked"].fillna("S",inplace=True)
test_data['Embarked'].fillna("S",inplace = True)

### 特征选择

In [25]:
#  特征选择
features = ["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
train_features = train_data[features]
train_labels =train_data['Survived']
test_features = test_data[features]

In [26]:
#  对于有固定的类的数据比如SEX，我们可以使用sklearn特征选择中的DictVectorizer类，
# 用它将可以处理符号化的对象，将符号转化成数字0/1进行表示
from sklearn.feature_extraction import DictVectorizer
dvec = DictVectorizer(sparse = False)

train_features = dvec.fit_transform(train_features.to_dict(orient = 'record'))
print(dvec.feature_names_)

['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']


In [27]:
# 使用ID3算法
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(train_features,train_labels)

test_features = dvec.transform(test_features.to_dict(orient ='record'))
# 决策树预测(test.csv中没有幸存者的信息)
pred_labels = clf.predict(test_features)

#  只能使用训练集中的数据进行模型评估，使用决策树自带的score函数进行验证
acc_decision_tree = round(clf.score(train_features,train_labels),6)
print("score的准确率%.4lf"%acc_decision_tree)

score的准确率0.9820


### K折交叉验证

> 交叉验证是一种常见的验证分类准确率的方法，原理是拿出大部分样本进行训练，少量的用于分类器的验证。K 折交叉验证，就是做 K 次交叉验证，每次选取 K 分之一的数据作为验证，其余作为训练。轮流 K 次，取平均值

> 在 sklearn 的 model_selection 模型选择中提供了 cross_val_score 函数。
cross_val_score 函数中的参数 cv 代表对原始数据划分成多少份，也就是我们的 K 值，一
般建议 K 值取 10，因此我们可以设置 CV=10

In [28]:
import numpy as np
from sklearn.model_selection import cross_val_score

# 使用K折交叉验证，统计决策树准确率
print("cross_val_score准确率：%.4lf"%np.mean(cross_val_score(clf,
                                                         train_features,
                                                         train_labels,
                                                        cv =10)))

cross_val_score准确率：0.7802


### 模型使用技巧总结

> - 特征选择是分类模型好坏我的关键，通常情况下，特征值不都是数值类型，可以使用DictVectorizer类进行转化
- 模型准确率需要考虑是否有测试集的实际结果作为对比，当测试集没有真实结果可以对比时，需要使用K折交叉验证cross_val_score